In [1]:
import os,sys, json,re, pickle
import magic, hashlib,  traceback ,ntpath, collections ,lief
from capstone import *
from capstone.x86 import *
import torch.nn as nn
import lief
from elftools.elf.elffile import ELFFile
from transformers import AdamW,AutoTokenizer
from tqdm import tqdm  # for our progress bar
from sklearn.metrics import precision_recall_fscore_support , accuracy_score,f1_score, confusion_matrix,mean_squared_error, mean_absolute_error, r2_score
from numpy import *
from num2words import num2words
import pandas as pd

In [2]:
ground_truth_json_dir = "/home/raisul/ANALYSED_DATA/ghidra_x86_pe_msvc_O2_static/"
#   
predict_json_dir = "/home/raisul/ANALYSED_DATA/ghidra_x86_pe_msvc_O2_static_stripped/"  #'/home/raisul/ANALYSED_DATA/prob_disasm_x86_pe_msvc_O2_static/'
BIN_FILE_TYPE = "PE"
BIN_PATH = "/home/raisul/DATA/x86_pe_msvc_O2_static_stripped/"

In [3]:

def get_offsets(json_file_path):
    with open(json_file_path, "r") as file:
        data = json.load(file)
    return list(map(int,list(data.keys())))



def get_instructions(bin_file_path  ):
    md = Cs(CS_ARCH_X86, CS_MODE_64)
    md.detail = True
    offset_inst = {}

    with open(bin_file_path, 'rb') as f:
        try:
            if BIN_FILE_TYPE == "ELF":
                elffile = ELFFile(f)
                textSection = elffile.get_section_by_name('.text').data()
                text_section_offset = elffile.get_section_by_name('.text')['sh_offset']
              
            elif BIN_FILE_TYPE == "PE":
        
                        
                pe_file = lief.parse(bin_file_path)
                text_section = pe_file.get_section(".text")
                text_section_offset = text_section.pointerto_raw_data
                textSection = bytes(text_section.content)
                
        except Exception as e:
                print("An error occurred:", e ,bin_file_path)
                return
    
        for byte_index in range(len(textSection)):
            try:    
    
                instruction = next(md.disasm(textSection[byte_index: byte_index+15 ], text_section_offset + byte_index ), None)
                offset_inst[text_section_offset+byte_index] = instruction
                
                # if instruction:
                #     print("%d:\t%s\t%s _\t%x" %(int(instruction.address), instruction.mnemonic, instruction.op_str, instruction.size))
                # else:
                #     print("%d:\t%s " % (text_section_offset + byte_index  , 'invalid instruction') )
    
            except Exception as e:
                print(traceback.print_exc() )
                print(e)
    return offset_inst

ground_truth__jsons  =  os.listdir(ground_truth_json_dir) 
ghidra_predict_jsons    =  os.listdir(predict_json_dir)


common_json_files = list(set(ground_truth__jsons) & set(ghidra_predict_jsons))
print(len(common_json_files))
common_json_files = common_json_files[0:1]
# common_json_files =['370cf902ac71f426c14725a24959e8de.json']
all_TP= all_FP= all_FN= all_TN = 0

for json_file_name in common_json_files:
    # print('\n\n\n',json_file_name)
    ground_truth_json_path =  os.path.join(ground_truth_json_dir, json_file_name)
    ghidra_predict_json_path =  os.path.join(predict_json_dir, json_file_name)
    ground_truth_offsets = get_offsets(ground_truth_json_path)
    predict_offsets = get_offsets(ghidra_predict_json_path)
    
    ground_truth_offsets_set = set(ground_truth_offsets)
    predict_offsets_set = set(predict_offsets)

    
    superset = set(range( min(ground_truth_offsets) , max(ground_truth_offsets) ) )
    
    true_positive = len(ground_truth_offsets_set&predict_offsets_set)
    false_positive = len(predict_offsets_set - ground_truth_offsets_set)
    false_negative = len( ground_truth_offsets_set -predict_offsets_set )
    true_negative = len( superset - ( ground_truth_offsets_set|predict_offsets_set)  )
    
    all_TP+=true_positive
    all_FP+=false_positive
    all_FN+=false_negative
    all_TN+=true_negative
    
    print('\ntrue_positive: ',true_positive , '\n false_positive: ',false_positive ,'\nfalse_negative:', false_negative , '\ntrue_negative: ',true_negative)

    
    bin_file_path = os.path.join(os.path.join(BIN_PATH ,  json_file_name.split('.')[0] ),  (json_file_name.split('.')[0]+ ('.exe' if BIN_FILE_TYPE == "PE" else '' )))
    exhaustive_disasm = get_instructions(bin_file_path)
    # print(exhaustive_disasm)
    if exhaustive_disasm:
      for offset in exhaustive_disasm:
          hash = '#' if offset in ground_truth_offsets else ' '
          dollar = '$' if offset in predict_offsets else ' '

          # if hash=='#' or dollar=='$':
          #     if exhaustive_disasm[offset].mnemonic!= 'int3':
          #         # pass
          #         print( hash , dollar , hex(offset), exhaustive_disasm[offset].mnemonic+ exhaustive_disasm[offset].op_str) #insn.mnemonic} {insn.op_str
          # if offset in predict_offsets and offset not in ground_truth_offsets :
          #     print('_'*130 , '>>')

          if(exhaustive_disasm[offset]):
              print(  hex(offset), " : ", exhaustive_disasm[offset].mnemonic+ " "+exhaustive_disasm[offset].op_str ,exhaustive_disasm[offset].size  ,hash , dollar, ) #insn.mnemonic} {insn.op_str
          else: 
              print(  hex(offset), " : ", 'Invalid ' ,hash , dollar, )
    else:
        print('exhaustive diasm none')
    # print(len(ground_truth_offsets) , len(predict_offsets))
    # false_negatives = list(set(ground_truth_offsets) - set(predict_offsets))
    



456962

true_positive:  1292 
 false_positive:  3 
false_negative: 191 
true_negative:  14759
0x400  :  int3  1    
0x401  :  int3  1    
0x402  :  int3  1    
0x403  :  int3  1    
0x404  :  int3  1    
0x405  :  jmp 0xec8 5 #  
0x406  :  mov esi, 0xe900000a 5    
0x407  :  or al, byte ptr [rax] 2    
0x408  :  add byte ptr [rax], al 2    
0x409  :  add cl, ch 2    
0x40a  :  jmp 0x18a0 5 # $
0x40b  :  xchg ecx, eax 1    
0x40c  :  adc al, 0 2    
0x40d  :  add byte ptr [rax], al 2    
0x40e  :  add cl, ch 2    
0x40f  :  jmp 0x7c0 5 # $
0x410  :  lodsb al, byte ptr [rsi] 1    
0x411  :  add eax, dword ptr [rax] 2    
0x412  :  add byte ptr [rax], al 2    
0x413  :  add cl, ch 2    
0x414  :  jmp 0x19b0 5 # $
0x415  :  xchg edi, eax 1    
0x416  :  adc eax, 0x2ee90000 5    
0x417  :  add byte ptr [rax], al 2    
0x418  :  add cl, ch 2    
0x419  :  jmp 0x104c 5 #  
0x41a  :  or al, 0 3    
0x41b  :  or al, 0 2    
0x41c  :  add byte ptr [rax], al 2    
0x41d  :  add cl, ch 2    
0x41e

In [4]:
print(f"TP: {all_TP}, FP: {all_FP}, FN: {all_FN}, TN: {all_TN}")


TP: 1292, FP: 3, FN: 191, TN: 14759


In [5]:
accuracy = (all_TP + all_TN) / (all_TP + all_TN + all_FP + all_FN)
precision = all_TP / (all_TP + all_FP) if (all_TP + all_FP) > 0 else 0
recall = all_TP / (all_TP + all_FN) if (all_TP + all_FN) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1_score:.4f}")

Accuracy: 0.9881
Precision: 0.9977
Recall: 0.8712
F1-score: 0.9302


In [6]:
def clip_to_unit_range(x):
    return max(0.0, min(1.0, x))

# Example usage
inputs = [-0.01, 0.0, 0.5, 1.0, 1.1]
outputs = [ 1-clip_to_unit_range(x) for x in inputs]
outputs2 = [ 1-x for x in inputs]
print(inputs)
print(outputs)  
print(outputs2)

TypeError: 'float' object cannot be interpreted as an integer

In [ ]:
for j in outputs2:
    print(j,.8*j)